In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


####  #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AndersonCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#  update with your username and password and CRUD Python module name

username = "aacuser"
password = "Animal123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("Project_Two_Anderson")

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the line below into the app.layout code according to your design
# Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data.image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Anderson Project Two Dashboard'))),
    html.Center(html.P('Select Option')),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DRIT'},
                {'label': 'RESET', 'value' : 'RESET'},
            ],
            value = 'RESET'   
        )
    ),
    html.Hr(),
    html.Center(html.P('Steven Anderson')),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#Steven Anderson

#Dasboard--------------------------------------------------------------------------------------
@app.callback(Output('datatable-id','data'),
              Output('datatable-id','columns'),
              [Input('filter-type','value')])

def update_dashboard(filter_type):
##  Add code to filter interactive data table with MongoDB queries
        if filter_type == 'RESET':
            df = pd.DataFrame.from_records(db.read({}))
        elif filter_type == 'WR':
            df = pd.DataFrame(list(db.read({'$and': [{"sex_upon_outcome" : "Intact Female"},
                                                     {'$or': [
                                                         {"breed": 'Labrador Retriever Mix'},
                                                         {"breed": 'Chesapeake Bay Retriever'},
                                                         {"breed": 'Newfoundland'}]},
                                                     {'$and': [{"age_upon_outcome_in_weeks" : {"$lte" : 156, "$gte" : 26}}]}
                                                    ]
                                           })))
            
        elif filter_type == 'MWR':
            df = pd.DataFrame(list(db.read({'$and': [{"sex_upon_outcome" : "Intact Male"},
                                                     {'$or': [
                                                         {"breed": 'German Shepherd'},
                                                         {"breed": 'Alaskan Malamute'},
                                                         {"breed": 'Old English Sheepdog'},
                                                         {"breed": 'Siberian Husky'},
                                                         {"breed": 'Rottweiler'}]},
                                                     {'$and': [{"age_upon_outcome_in_weeks" : {"$lte" : 156, "$gte" : 26}}]}
                                                    ]
                                           })))
            
        elif filter_type == 'DRIT':
            df = pd.DataFrame(list(db.read({'$and': [{"sex_upon_outcome" : "Intact Male"},
                                                     {'$or': [
                                                         {"breed": 'German Shepherd'},
                                                         {"breed": 'Doberman Pinscher'},
                                                         {"breed": 'Golden Retriever'},
                                                         {"breed": 'Bloodhound'},
                                                         {"breed": 'Rottweiler'}]},
                                                     {'$and': [{"age_upon_outcome_in_weeks" : {"$lte" : 300, "$gte" : 26}}]}
                                                    ]
                                           })))

        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
       
       
        return (data,columns)
    
##Style---------------------------------------------------------------------------------------------
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#Pie Chart------------------------------------------------------------------------------------------
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):   
    dff = pd.DataFrame.from_dict(viewData)
    x = []
    return [dcc.Graph(figure = px.histogram(dff, x = 'breed'))]

#Geo location-----------------------------------------------------------------------------------------
# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows",)])

def update_map(viewData, index):    
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center = [30.75, -97.48], zoom = 10, children = [
                  dl.TileLayer(id="base-layer-id"),
                  dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                           children = [
                               dl.Tooltip(dff.iloc[row,4]),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(dff.iloc[row,9])
                               ])
                           ])
              ])
    ]



app.run_server(mode='external', debug=True, port=30035)


Connected test
Dash app running on http://127.0.0.1:30035/
